In [1]:
import mlflow

In [5]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/5', creation_time=1723148162406, experiment_id='5', last_update_time=1723148162406, lifecycle_stage='active', name='best-model', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/2', creation_time=1723144855817, experiment_id='2', last_update_time=1723144855817, lifecycle_stage='active', name='lgbm-best-model', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/1', creation_time=1723144092599, experiment_id='1', last_update_time=1723144092599, lifecycle_stage='active', name='compare-models', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1723143646452, experiment_id='0', last_update_time=1723143646452, lifecycle_stage='active', name='Default', tags={}>]

In [22]:
runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse_lgbm  < 0.885",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse_lgbm ASC"]
)

In [24]:
runs[0]

<Run: data=<RunData: metrics={'rmse_lgbm': 0.8806487710167833}, params={'model': 'lgbm',
 'test-data-path': '../data/london_weather.csv',
 'train-data-path': '../data/london_weather.csv'}, tags={'developer': 'hema',
 'mlflow.runName': 'capricious-roo-899',
 'mlflow.source.name': '/home/ubuntu/.local/share/virtualenvs/mlops_temperature_prediction-M3ZLPW1f/lib/python3.9/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'ubuntu'}>, info=<RunInfo: artifact_uri='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/2/4b25ddb157ec403cad3cfb6bc2b6e3e4/artifacts', end_time=1723150622180, experiment_id='2', lifecycle_stage='active', run_id='4b25ddb157ec403cad3cfb6bc2b6e3e4', run_name='capricious-roo-899', run_uuid='4b25ddb157ec403cad3cfb6bc2b6e3e4', start_time=1723150620293, status='FINISHED', user_id='ubuntu'>, inputs=<RunInputs: dataset_inputs=[]>>

In [26]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse_lgbm']:.4f}")

run id: 4b25ddb157ec403cad3cfb6bc2b6e3e4, rmse: 0.8806
run id: facdad167b9e4ac6b7c4956f54244f3b, rmse: 0.8811
run id: 3624c85848dc4064ab91c77e0421c744, rmse: 0.8820
run id: 5cd80a90c56e40079cf29daeccaa59ba, rmse: 0.8820
run id: 4b7b9a06e67c4843bd336526d442f9a4, rmse: 0.8823


In [39]:
runs = client.search_runs( 
    experiment_ids='6',
    filter_string="metrics.rmse_lgbm  < 0.885",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse_lgbm ASC"]
)

In [40]:
runs[0]

<Run: data=<RunData: metrics={'rmse_lgbm': 0.8806487710167833}, params={'model': 'lgbm',
 'test-data-path': '../data/london_weather.csv',
 'train-data-path': '../data/london_weather.csv'}, tags={'developer': 'hema',
 'mlflow.log-model.history': '[{"run_id": "8008366d93cc4bdfbcd135dc1d6e50ff", '
                             '"artifact_path": "lgbm_reg", "utc_time_created": '
                             '"2024-08-09 05:30:57.662417", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.19", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.0.2", '
                             '"serialization_format": 

In [43]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse_lgbm']:.4f}")

run id: 8008366d93cc4bdfbcd135dc1d6e50ff, rmse: 0.8806


In [45]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [46]:
run_id = "8008366d93cc4bdfbcd135dc1d6e50ff"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="london-temperature-lgbm-regressor")

Registered model 'london-temperature-lgbm-regressor' already exists. Creating a new version of this model...
Created version '1' of model 'london-temperature-lgbm-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1723182544091, current_stage='None', description=None, last_updated_timestamp=1723182544091, name='london-temperature-lgbm-regressor', run_id='8008366d93cc4bdfbcd135dc1d6e50ff', run_link=None, source='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/6/8008366d93cc4bdfbcd135dc1d6e50ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [47]:
model_name = "london-temperature-lgbm-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [48]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1723182544091, current_stage='Staging', description=None, last_updated_timestamp=1723182889548, name='london-temperature-lgbm-regressor', run_id='8008366d93cc4bdfbcd135dc1d6e50ff', run_link=None, source='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/6/8008366d93cc4bdfbcd135dc1d6e50ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [49]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1723182544091, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-08-09', last_updated_timestamp=1723182982940, name='london-temperature-lgbm-regressor', run_id='8008366d93cc4bdfbcd135dc1d6e50ff', run_link=None, source='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/6/8008366d93cc4bdfbcd135dc1d6e50ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
## TO BE DONE. RUN THE MODEL ON NEW DATA